In [ ]:
# make sure to keep this notebook just outside the built darknet directory

import os
!pip install kagglehub --quiet
import kagglehub
dataset_path = "/present/working/directory/HRSC2016-MS/"

In [ ]:
def download_and_extract_dataset():
    print("Downloading HRSC2016-MS dataset from Kaggle Hub...")
    path = kagglehub.dataset_download("weiming97/hrsc2016-ms-dataset")
    print("Path to dataset files:", path)

    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)
        os.system(f"cp -r {path}/* {dataset_path}")

In [ ]:
download_and_extract_dataset()

In [ ]:
# parse the dataset xml's annotations and store them in .txt files

import os
!pip install tqdm
!pip install pillow
from tqdm import tqdm
from PIL import Image
import xml.etree.ElementTree as ET

def make_yolo_dir(cwd):
    yolo_dir = os.path.join(cwd, 'HRSC-YOLO')
    os.mkdir(yolo_dir)
    for i in ['train', 'val', 'test']:
        folder = os.path.join(yolo_dir, i)
        os.mkdir(folder)
        for j in ['images', 'labels']:
            subfolder = os.path.join(folder, j)
            os.mkdir(subfolder)
    return yolo_dir

def xml2txt(xml_file_path, txt_file, w, h):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    objs = root.findall('object')

    if not objs:
        print(f"Warning: No objects found in {xml_file_path}. Creating an empty annotation file.")
        with open(txt_file, 'w') as f:
            f.write("\n")  # Empty file for compatibility
        return

    with open(txt_file, 'w') as f:
        for obj in objs:
            try:
                xmin = int(obj.find('bndbox/xmin').text)
                ymin = int(obj.find('bndbox/ymin').text)
                xmax = int(obj.find('bndbox/xmax').text)
                ymax = int(obj.find('bndbox/ymax').text)

                # Convert to YOLO format (normalized values)
                x_center = (xmin + xmax) / (2.0 * w)
                y_center = (ymin + ymax) / (2.0 * h)
                box_w = (xmax - xmin) / w
                box_h = (ymax - ymin) / h

                # Save in YOLO format
                f.write(f"0 {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")
            except Exception as e:
                print(f"Error processing {xml_file_path}: {e}")



def read_list_file(cwd):
    with open(os.path.join(cwd, 'ImageSets/train.txt'), 'r') as f:
        train_list = f.read().splitlines()
    with open(os.path.join(cwd, 'ImageSets/val.txt'), 'r') as f:
        val_list = f.read().splitlines()
    with open(os.path.join(cwd, 'ImageSets/test.txt'), 'r') as f:
        test_list = f.read().splitlines()
    return train_list, val_list, test_list

def construct_path(file_name, cwd, yolo_dir, mode):
    if mode == 'train':
        train_file_path = os.path.join(cwd, f'AllImages/{file_name}.bmp')
        train_xml_path = os.path.join(cwd, f'Annotations/{file_name}.xml')
        save_txt_path = os.path.join(yolo_dir, f'train/labels/{file_name}.txt')
        save_png_path = os.path.join(yolo_dir, f'train/images/{file_name}.png')
    elif mode == 'val':
        train_file_path = os.path.join(cwd, f'AllImages/{file_name}.bmp')
        train_xml_path = os.path.join(cwd, f'Annotations/{file_name}.xml')
        save_txt_path = os.path.join(yolo_dir, f'val/labels/{file_name}.txt')
        save_png_path = os.path.join(yolo_dir, f'val/images/{file_name}.png')
    elif mode == 'test':
        train_file_path = os.path.join(cwd, f'AllImages/{file_name}.bmp')
        train_xml_path = os.path.join(cwd, f'Annotations/{file_name}.xml')
        save_txt_path = os.path.join(yolo_dir, f'test/labels/{file_name}.txt')
        save_png_path = os.path.join(yolo_dir, f'test/images/{file_name}.png')
    else:
        print(f"Unrecognized mode {mode}!")
    return train_file_path, train_xml_path, save_txt_path, save_png_path


def parser():
    cwd = os.getcwd()
    yolo_dir = make_yolo_dir(cwd)

    #yolo_dir = "/content/HRSC-YOLO"
    train_list, val_list, test_list = read_list_file("/home/opencvuniv/Work/Shubham/install_darknet/HRSC2016-MS/")

    for train_file in tqdm(train_list):
        bmp_path, xml_path, txt_path, png_path = construct_path(train_file, "/home/opencvuniv/Work/Shubham/install_darknet/HRSC2016-MS/", yolo_dir, 'train')

        img = Image.open(bmp_path)
        w, h = img.size
        xml2txt(xml_path, txt_path, w, h)

        img.save(png_path, format='PNG')

    for val_file in tqdm(val_list):
        bmp_path, xml_path, txt_path, png_path = construct_path(val_file, "/home/opencvuniv/Work/Shubham/install_darknet/HRSC2016-MS/", yolo_dir, 'val')

        img = Image.open(bmp_path)
        w, h = img.size
        xml2txt(xml_path, txt_path, w, h)

        img.save(png_path, format='PNG')

    for test_file in tqdm(test_list):
        bmp_path, xml_path, txt_path, png_path = construct_path(test_file, "/home/opencvuniv/Work/Shubham/install_darknet/HRSC2016-MS/", yolo_dir, 'test')

        img = Image.open(bmp_path)
        w, h = img.size
        xml2txt(xml_path, txt_path, w, h)

        img.save(png_path, format='PNG')

In [ ]:
parser()

In [ ]:
!pwd

In [ ]:
import shutil
import random

source_dir = "/present/working/directory/HRSC-YOLO/"
dest_dir = "/present/working/directory/experimentals/"

# Create the 'experimentals' directory with 'images' and 'labels' subdirectories

os.makedirs('experimentals', exist_ok=True)
os.makedirs(os.path.join(dest_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "labels"), exist_ok=True)

# Define dataset splits
splits = ['train', 'val', 'test']

# Copy all images and labels from HRSC-YOLO to experimentals
def copy_files():
    for split in splits:
        image_source = os.path.join(source_dir, split, "images")
        label_source = os.path.join(source_dir, split, "labels")
        
        for file in os.listdir(image_source):
            src_path = os.path.join(image_source, file)
            dest_path = os.path.join(dest_dir, "images", file)
            shutil.copy2(src_path, dest_path)
        
        for file in os.listdir(label_source):
            src_path = os.path.join(label_source, file)
            dest_path = os.path.join(dest_dir, "labels", file)
            shutil.copy2(src_path, dest_path)

copy_files()
print("All images and labels copied successfully to experimentals/")

In [ ]:
# Define paths
base_dir = "/present/working/directory/experimentals"
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

# Output directories
output_dirs = {
    "train": "train",
    "val": "val",
    "test": "test"
}

# Create train, val, and test directories with images and labels subfolders
for split in output_dirs.values():
    os.makedirs(os.path.join(split, "images"), exist_ok=True)
    os.makedirs(os.path.join(split, "labels"), exist_ok=True)

# Get all image filenames
image_files = [f for f in os.listdir(images_dir) if f.endswith(".png")]
random.shuffle(image_files)  # Shuffle dataset

# Define split sizes
train_size, val_size, test_size = 610, 460, 610

# Split dataset
train_images = image_files[:train_size]
val_images = image_files[train_size:train_size + val_size]
test_images = image_files[train_size + val_size:train_size + val_size + test_size]

# Function to copy files
def copy_files2(image_list, split):
    for img_name in image_list:
        label_name = img_name.replace(".png", ".txt")  # Corresponding label
        shutil.copy(os.path.join(images_dir, img_name), os.path.join(split, "images", img_name))
        shutil.copy(os.path.join(labels_dir, label_name), os.path.join(split, "labels", label_name))

# Copy files to respective folders
copy_files2(train_images, "train")
copy_files2(val_images, "val")
copy_files2(test_images, "test")

print("Dataset split and copied successfully!")

In [ ]:
import os
import shutil

source_root = "/present/working/directory"
dest_root = "present/working/directory"

# for creating text files having images and their corresponding labels together to be used by the Darknet
def copy_files3(source_root, dest_root):
    for split in ['train', 'test', 'val']:
        src_image_dir = os.path.join(source_root, split, 'images')
        src_label_dir = os.path.join(source_root, split, 'labels')

        dest_dir = os.path.join(dest_root, f"{split}2")
        os.makedirs(dest_dir, exist_ok=True)

        if not os.path.exists(src_image_dir) or not os.path.exists(src_label_dir):
            print(f"Skipping missing directory: {src_image_dir} or {src_label_dir}")
            continue

        for filename in os.listdir(src_image_dir):
            src_image_path = os.path.join(src_image_dir, filename)
            src_label_path = os.path.join(src_label_dir, filename.replace('.png', '.txt'))

            if os.path.isfile(src_image_path):
                shutil.copy(src_image_path, dest_dir)
            if os.path.isfile(src_label_path):
                shutil.copy(src_label_path, dest_dir)

copy_files3(source_root, dest_root)
print("Copying completed successfully!")

In [ ]:
!nvidia-smi

In [ ]:
%cd /present/working/directory/darknet/
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov7x.conv.147
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov7x.weights

In [ ]:
%%writefile prepare_darknet_image_txt_paths.py
import os

DATA_ROOT_TRAIN = os.path.join(
    '/directory/path/just/outside/the/built/darknet', 'train2'
)
DATA_ROOT_VALID = os.path.join(
    '/directory/path/just/outside/the/built/darknet', 'val2'
)

DATA_ROOT_TEST = os.path.join(
    '/directory/path/just/outside/the/built/darknet', 'test2'
)

train_image_files_names = os.listdir(os.path.join(DATA_ROOT_TRAIN))
with open('train.txt', 'w') as f:
    for file_name in train_image_files_names:
        if not '.txt' in file_name:
            write_name = os.path.join(DATA_ROOT_TRAIN, file_name)
            f.writelines(write_name+'\n')

valid_data_files__names = os.listdir(os.path.join(DATA_ROOT_VALID))
with open('valid.txt', 'w') as f:
    for file_name in valid_data_files__names:
        if not '.txt' in file_name:
            write_name = os.path.join(DATA_ROOT_VALID, file_name)
            f.writelines(write_name+'\n')

test_data_files__names = os.listdir(os.path.join(DATA_ROOT_TEST))
with open('test.txt', 'w') as f:
    for file_name in test_data_files__names:
        if not '.txt' in file_name:
            write_name = os.path.join(DATA_ROOT_TEST, file_name)
            f.writelines(write_name+'\n')

In [ ]:
!python prepare_darknet_image_txt_paths.py

In [ ]:
%%writefile cfg/yolov7-darknet-hrsc2016-ms.cfg
[net]
# Testing
#batch=64
#subdivisions=64
# Training
batch=64
subdivisions=64
width=640
height=640
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.00261
burn_in=1000

max_batches = 2000
policy=steps
steps=1600,1800
scales=.1,.1


# 0
[convolutional]
batch_normalize=1
filters=40
size=3
stride=1
pad=1
activation=swish


# 1
[convolutional]
batch_normalize=1
filters=80
size=3
stride=2
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=80
size=3
stride=1
pad=1
activation=swish


# 3
[convolutional]
batch_normalize=1
filters=160
size=3
stride=2
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=swish

[route]
layers=-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=swish

[route]
layers = -1,-3,-5,-7,-9

# 14
[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish


[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=160
size=1
stride=1
pad=1
activation=swish

[route]
layers=-3

[convolutional]
batch_normalize=1
filters=160
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=160
size=3
stride=2
pad=1
activation=swish

# 20
[route]
layers = -1,-4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=swish

[route]
layers=-2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[route]
layers = -1,-3,-5,-7,-9

# 31
[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish


[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish

[route]
layers=-3

[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=320
size=3
stride=2
pad=1
activation=swish

# 37
[route]
layers = -1,-4

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=swish

[route]
layers=-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[route]
layers = -1,-3,-5,-7,-9

# 48
[convolutional]
batch_normalize=1
filters=1280
size=1
stride=1
pad=1
activation=swish


[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish

[route]
layers=-3

[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=640
size=3
stride=2
pad=1
activation=swish

# 54
[route]
layers = -1,-4

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=swish

[route]
layers=-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[route]
layers = -1,-3,-5,-7,-9

# 65
[convolutional]
batch_normalize=1
filters=1280
size=1
stride=1
pad=1
activation=swish

##################################

### SPPCSP ###
[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish

[route]
layers = -2

[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=640
activation=swish

[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish

### SPP ###
[maxpool]
stride=1
size=5

[route]
layers=-2

[maxpool]
stride=1
size=9

[route]
layers=-4

[maxpool]
stride=1
size=13

[route]
layers=-6,-5,-3,-1
### End SPP ###

[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=640
activation=swish

[route]
layers = -1, -13

# 80
[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish


[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish

[upsample]
stride=2

[route]
layers = 48

[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish

[route]
layers = -1,-3


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=swish

[route]
layers=-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[route]
layers = -1,-3,-5,-7,-9

# 96
[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish


[convolutional]
batch_normalize=1
filters=160
size=1
stride=1
pad=1
activation=swish

[upsample]
stride=2

[route]
layers = 31

[convolutional]
batch_normalize=1
filters=160
size=1
stride=1
pad=1
activation=swish

[route]
layers = -1,-3


[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=swish

[route]
layers=-2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=swish

[route]
layers = -1,-3,-5,-7,-9

# 112
[convolutional]
batch_normalize=1
filters=160
size=1
stride=1
pad=1
activation=swish


[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=160
size=1
stride=1
pad=1
activation=swish

[route]
layers=-3

[convolutional]
batch_normalize=1
filters=160
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=160
size=3
stride=2
pad=1
activation=swish

[route]
layers = -1,-4,96


[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=swish

[route]
layers=-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=swish

[route]
layers = -1,-3,-5,-7,-9

# 129
[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish


[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish

[route]
layers=-3

[convolutional]
batch_normalize=1
filters=320
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=320
size=3
stride=2
pad=1
activation=swish

[route]
layers = -1,-4,80


[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=swish

[route]
layers=-2

[convolutional]
batch_normalize=1
filters=512
size=1
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=swish

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=swish

[route]
layers = -1,-3,-5,-7,-9

# 146
[convolutional]
batch_normalize=1
filters=640
size=1
stride=1
pad=1
activation=swish

#############################

# ============ End of Neck ============ #

# ============ Head ============ #


# P3
[route]
layers = 112

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=320
activation=swish

[convolutional]
size=1
stride=1
pad=1
filters=18
#activation=linear
activation=logistic

[yolo]
mask = 0,1,2
anchors = 12,16, 19,36, 40,28, 36,75, 76,55, 72,146, 142,110, 192,243, 459,401
classes=1
num=9
jitter=.1
scale_x_y = 2.0
objectness_smooth=1
ignore_thresh = .7
truth_thresh = 1
#random=1
resize=1.5
iou_thresh=0.2
iou_normalizer=0.05
cls_normalizer=0.5
obj_normalizer=1.0
iou_loss=ciou
nms_kind=diounms
beta_nms=0.6
new_coords=1
max_delta=2


# P4
[route]
layers = 129

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=640
activation=swish

[convolutional]
size=1
stride=1
pad=1
filters=18
#activation=linear
activation=logistic

[yolo]
mask = 3,4,5
anchors = 12,16, 19,36, 40,28, 36,75, 76,55, 72,146, 142,110, 192,243, 459,401
classes=1
num=9
jitter=.1
scale_x_y = 2.0
objectness_smooth=1
ignore_thresh = .7
truth_thresh = 1
#random=1
resize=1.5
iou_thresh=0.2
iou_normalizer=0.05
cls_normalizer=0.5
obj_normalizer=1.0
iou_loss=ciou
nms_kind=diounms
beta_nms=0.6
new_coords=1
max_delta=2


# P5
[route]
layers = 146

[convolutional]
batch_normalize=1
size=3
stride=1
pad=1
filters=1280
activation=swish

[convolutional]
size=1
stride=1
pad=1
filters=18
#activation=linear
activation=logistic

[yolo]
mask = 6,7,8
anchors = 12,16, 19,36, 40,28, 36,75, 76,55, 72,146, 142,110, 192,243, 459,401
classes=1
num=9
jitter=.1
scale_x_y = 2.0
objectness_smooth=1
ignore_thresh = .7
truth_thresh = 1
#random=1
resize=1.5
iou_thresh=0.2
iou_normalizer=0.05
cls_normalizer=0.5
obj_normalizer=1.0
iou_loss=ciou
nms_kind=diounms
beta_nms=0.6
new_coords=1
max_delta=2

In [ ]:
%%writefile build/darknet/x64/data/ship.names
ship

In [ ]:
%%writefile build/darknet/x64/data/hrsc2016-ms-yolov7.data
classes = 1
train  = train.txt
valid  = valid.txt
names = build/darknet/x64/data/ship.names
backup = backup_2000

In [ ]:
%%writefile build/darknet/x64/data/ship_test.data
classes = 1
train  = train.txt
valid  = test.txt
names = build/darknet/x64/data/ship.names
backup = backup_test_2000

In [ ]:
os.makedirs('backup_2000', exist_ok=True)
print('Backup directory created for YOLOv7-based Darknet')

In [ ]:
os.makedirs('backup_test_2000', exist_ok=True)
print('Test Backup directory created for YOLOv7-based Darknet')

In [ ]:
# run the command in terminal to be opened in the built darknet directory itself
# to start the darknet training process

./darknet detector train build/darknet/x64/data/hrsc2016-ms-yolov7.data cfg/yolov7-darknet-hrsc2016-ms.cfg yolov7x.conv.147 -map -dont_show

In [ ]:
# run the command in terminal to be opened in the built darknet directory itself
# to get the mAP scores at various IoU thresholds

!./darknet detector map build/darknet/x64/data/ship_test.data cfg/yolov7-darknet-hrsc2016-ms.cfg backup_2000/yolov7-hrsc2016-ms-final.weights
!./darknet detector map build/darknet/x64/data/ship_test.data cfg/yolov7-darknet-hrsc2016-ms.cfg backup_2000/yolov7-hrsc2016-ms-final.weights -iou_thresh 0.75

In [ ]:
os.makedirs('backup_2000', exist_ok=True)
print('Directory to store detections on test images has been created')

In [ ]:
# add below 2 code lines in darknet_images.py script and then run the below command in terminal to see detections on test images
# place where to add these two lines has been shown in the article

output_image_path = os.path.join("yolov7_darknet_test_results", image_name.split(os.path.sep)[-1])
cv2.imwrite(output_image_path, image)

In [ ]:
# to visualize the detections on the test images

# we can visualize darknet detections on train as well as validation images, we just need to
# change the path of the images directory in the below command as well as we need to create 
# separate directories as shown in the last third command

# command to be executed in terminal

python darknet_images.py --data_file build/darknet/x64/data/ship_test.data --input /path/to/the/directory/containing/train/images --config_file cfg/yolov7-darknet-hrsc2016-ms.cfg --weights backup_2000/yolov7-darknet-hrsc2016-ms_final.weights --thresh 0.25 --ext_output --save_labels --dont_show 

In [ ]:
# to get the darknet detections as bounding boxes values

./darknet detector test build/darknet/x64/data/hrsc2016-ms-yolov7.data cfg/yolov7-darknet-hrsc2016-ms.cfg backup_2000/yolov7-darknet-hrsc2016-ms_final.weights -dont_show -ext_output < test.txt > results_darknet_test.txt